# Style transfer with VGG-19 architecture

In [0]:
# import resources
%matplotlib inline

from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.optim as optim
from torchvision import transforms, models

# Load in VGG19 (features)

VGG19 is split into two portions

* vgg19.features, which are all the convolutional and polling layers
* vgg19.classifier, which are the three linear classifier layers at the end

We only need the features portion, which we're going to load in and "freeze" the weights of, below.

In [0]:
# get the features portion
vgg = models.vgg19(pretrained=True).features

# freeze all VGG parameters since we're only optimizing the target image
for param in vgg.parameters():
  param.requires_grad_(False)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/checkpoints/vgg19-dcbb9e9d.pth


In [0]:
#move the model to GPU, if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vgg.to(device)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

# Load in Content and Style images


In [0]:
def load_image(img_path, max_size = 400, shape=None):
    ''' Load in and transform an image, making sure the image is <= 400 
    pixels in the x-y dims.'''

    image = Image.open(img_path).convert('RGB')

    # large images will slow down processing
    if max(image.size) > max_size:
      size = max_size
    else:
      size = max(image.size)

    if shape is not None:
      size = shape

    in_transform = transforms.Compose([
                                       transforms.Resize(size),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.485, 0.456, 0.406),
                                                            (0.229, 0.224, 0.225))])
    # discard the transparent, alpha channel (that's the :3) and add the bath dimension
    image = in_transform(image)[:3, :, :].unsqueeze(0)
    return image

loading images by file name and forcing the style image to be the same size as the content image.

In [0]:
# load in content and style image
content = load_image('./content1.jpg').to(device)
# Resize style to match content, makes code easier
style = load_image('./style3.jpg', shape=content.shape[-2:]).to(device)

In [0]:
# helper function for un-normalizing an image
# and converting it from a tensor to a Numpy image for display
def im_convert(tensor):
  ''' Display a tensor as an image. '''

  image = tensor.to("cpu").clone().detach()
  image = image.numpy().squeeze()
  image = image.transpose(1,2,0)
  image = image * np.array((0.229, 0.224, 0.225)) + np.array((0.485, 0.456, 0.406))
  image = image.clip(0, 1)

  return image

In [0]:
# display the images
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 10))
# content and style img side-by-side
ax1.imshow(im_convert(content))
ax2.imshow(im_convert(style))

# VGG19 Layers

In [0]:
# print out VGG19 structure so you can see the names of various layers
print(vgg)

# Content and styles features

In [0]:
def get_features(image, model, layers=None):
  ''' Run an image forward through a model and get the features for
  a set of layers. Default layers are for VGGNet matching Gatys et al (2016)
  '''

  # Need the layers for the content and style representationss of an image
  if layers is None:
    layers = {'0': 'conv1_1',
              '5': 'conv2_1',
              '10': 'conv3_1',
              '19': 'conv4_1',
              '21': 'conv4_2',
              '28': 'conv5_1'} # content representation
  
  features = {}
  x = image
  # model.modules is a dictionary holding each module in the model
  for name, layer in model._modules.items():
    x = layer(x)
    if name in layers:
      features[layers[name]] = x

  return features       



# Gram Matrix
The output of every convolutional layer is a Tensor with dimensions associated with the batch_size, a depth, d and some height and width (h, w). The Gram matrix of a convolutional layer can be calculated as follows:

* Get the depth, height and width of a tensor using batch_size, d, h, w = tensor.size
* Reshape that tensor so that the spatial dimensions are flattened
* Calculate the gram matrix by multiplying the reshaped tensor by it's transpose

Note: you can multiply two matrices using torch.mm(matrix1, matrix2)

In [0]:
def gram_matrix(tensor):
  ''' Calculate the gram matrix of a given tensor
  Gram matrix: https://en.wikipedia.org/wiki/Gramian_matrix
  '''
  # get the batch size, depth,height, and width of the tensor
  _, d, h, w = tensor.size()

  # tensor so we're multiplying the features for each channel
  tensor = tensor.view(d, h * w)

  # calculate the gram matrix
  gram = torch.mm(tensor,tensor.t())

  return gram

# Putting it all Together


In [0]:
# get content and style features only once before forming the target image
content_features = get_features(content, vgg)
style_features = get_features(style, vgg)

# calculate the gram matrices for each layer of our style representation
style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

# create a third 'target' image and prep it for change
# it is a good idea to start of with the target as a copy of our content image
# then iteratively change its style
target = content.clone().requires_grad_(True).to(device)

# Loss and weights

In [0]:
# weights for each style layer 
# weighting earlier layers more will result in 'larger' style artifacts
# notice we are excluding 'conv4_2' our content representation
style_weights = {'conv1_1': 1.,
                 'conv2_1': 0.75,
                 'conv3_1': 0.2,
                 'conv4_1': 0.2,
                 'conv5_1': 0.2}

#you may choose to leave these as is
content_weight = 1 # alpha
style_weight = 1e6 # beta

# Updating the target & calculating losses

In [0]:
# for displaying the target image intermittently
show_every = 400

# iteration hyperparameters
optimizer = optim.Adam([target], lr= 0.03)
steps = 2000 # decide how many iterations to update your image (5000)

for ii in range(1, steps+1):

  # get the features to target image
  target_features = get_features(target, vgg)
  # the content loss
  content_loss = torch.mean((target_features['conv4_2'] - content_features['conv4_2'])**2)

  # the style loss
  # initialize the style loss to 0
  style_loss = 0
  # iterate through each style layer and add to the style loss
  for layer in style_weights:
    # get the 'target' style representation for the layer
    target_feature = target_features[layer]
    # Calculate the target gram matrix
    target_gram = gram_matrix(target_feature)
    _, d, h, w = target_feature.shape
    # get the style representation
    style_gram = style_grams[layer]
    # Calculate the style loss for one layer, wighted appropriately
    layer_style_loss = style_weights[layer] * torch.mean((target_gram - style_gram)**2)
    # add the style loss
    style_loss += layer_style_loss/(d * h * w)
  
  # Calculate the total loss
  total_loss = content_weight * content_loss + style_weight * style_loss
  
  # update your target image
  optimizer.zero_grad()
  total_loss.backward()
  optimizer.step()

  # display intermediate images and print the loss
  if ii % show_every == 0:
    print('Total loss: ', total_loss.item())
    plt.imshow(im_convert(target))
    plt.show()



# Display the target image

In [0]:
# display content and final target image
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 10))
ax1.imshow(im_convert(content))
ax2.imshow(im_convert(target))